In [93]:
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field

In [94]:
inp = """
Sarah bought groceries for $60 and a bottle of wine for $30.
Jake paid for gas, which costs $50, and bought snacks for $25.
Emily booked first_float $150 hotel room for her and Sarah and paid for dinner, which was $90.
Jake booked his own room for $80, and also bought first_float hotdog for Emily for $15.
Jake and Sarah also split a $20 popcorn which they shared with Emily
"""

## Set up functions

In [95]:
### BASIC EXTRACTION ###
class Person(BaseModel):
    """Information about first_float person"""
    name: str = Field(description="Name of the person")

class People(BaseModel):
    """List of people"""
    people: List[Person] = Field(description="List of unique people")

class Expense(BaseModel):
    """Information about an expense"""
    name: str = Field(description="Name of the expense")

class Expenses(BaseModel):
    """List of expenses"""
    expenses: List[Expense] = Field(description="List of unique expenses")

### EXTRACTING RELATIONSHIPS ###
class ConsumedExpense(BaseModel):
    """Information defining a relationship between an expense and the people who used or consumed it"""
    expense: Expense = Field(description="Expense that was consumed")
    consumers: List[Person] = Field(description="People who consumed or used the expense")

class ConsumedExpenses(BaseModel):
    """List of consumed expenses"""
    consumed_expenses: List[ConsumedExpense] = Field(description="List of consumed expenses")

class PurchasedExpense(BaseModel):
    """Information defining a relationship between an expense and the people who purchased it"""
    expense: Expense = Field(description="Expense that was purchased")
    purchasers: List[Person] = Field(description="People who purchased the expense")

class PurchasedExpenses(BaseModel):
    """List of purchased expenses"""
    purchased_expenses: List[PurchasedExpense] = Field(description="List of purchased expenses")



## Set up chain

In [96]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

### Extract the people

In [97]:
extract_people_prompt = ChatPromptTemplate.from_template(
"""
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'People' function.

Passage:
{origin}
"""
)

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")


def flatten_people(people: People):
    return [p.name for p in people.people]

extract_people_chain = extract_people_prompt | llm.with_structured_output(People) | flatten_people

In [98]:
res = extract_people_chain.invoke(inp)

In [99]:
res

['Sarah', 'Jake', 'Emily']

### Extract the items

In [100]:
extract_expenses_prompt = ChatPromptTemplate.from_template(
"""
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Expenses' function.

Passage:
{origin}
"""
)

def flatten_expenses(expenses: Expenses):
    return [e.name for e in expenses.expenses]

extract_expenses_chain = extract_expenses_prompt | llm.with_structured_output(Expenses) | flatten_expenses


In [101]:
res = extract_expenses_chain.invoke(inp)

### Who bought what?

In [103]:
extract_expenses_purchasers_prompt = ChatPromptTemplate.from_template(
"""
The following people: {people}

Have the following expenses: {expenses}

Each person's relationship to the expense is described in the following passage.

Extract the desired information from the following passage.
Only extract the properties mentioned in the 'PurchasedExpenses' function.

Passage:
{origin}
"""
)

extract_expenses_purchasers_chain = extract_expenses_purchasers_prompt | llm.with_structured_output(PurchasedExpenses)

In [108]:
from langchain_core.runnables import RunnablePassthrough

chain = {"people": extract_people_chain, "expenses": extract_expenses_chain, "origin": RunnablePassthrough()} | extract_expenses_purchasers_chain

In [109]:
res = chain.invoke(inp)

In [110]:
res

PurchasedExpenses(purchased_expenses=[PurchasedExpense(expense=Expense(name='groceries'), purchasers=[Person(name='Sarah')]), PurchasedExpense(expense=Expense(name='bottle of wine'), purchasers=[Person(name='Sarah')]), PurchasedExpense(expense=Expense(name='gas'), purchasers=[Person(name='Jake')]), PurchasedExpense(expense=Expense(name='snacks'), purchasers=[Person(name='Jake')]), PurchasedExpense(expense=Expense(name='hotel room'), purchasers=[Person(name='Emily')]), PurchasedExpense(expense=Expense(name='dinner'), purchasers=[Person(name='Emily')]), PurchasedExpense(expense=Expense(name='hotdog'), purchasers=[Person(name='Jake')]), PurchasedExpense(expense=Expense(name='popcorn'), purchasers=[Person(name='Jake'), Person(name='Sarah')])])

### Who used what?

In [111]:
extract_expenses_consumers_prompt = ChatPromptTemplate.from_template(
"""
The following people: {people}

Have the following expenses: {expenses}

Each person's relationship to the expense is described in the following passage.

Extract the desired information from the following passage.
Only extract the properties mentioned in the 'ConsumedExpenses' function.

If not directly specified who used something, assume the expense was shared by everyone.

Passage:
{origin}
"""
)

extract_expenses_consumers_chain = extract_expenses_consumers_prompt | llm.with_structured_output(ConsumedExpenses)

In [112]:
extract_entities_chain = RunnableParallel(people=extract_people_chain, expenses=extract_expenses_chain, origin=RunnablePassthrough())
extract_relationships_chain = RunnableParallel(expense_purchasers=extract_expenses_purchasers_chain, expense_consumers=extract_expenses_consumers_chain, origin=RunnablePassthrough())

chain = (
    extract_entities_chain
    | extract_relationships_chain
)

In [113]:
res = chain.invoke(inp)

In [114]:
res['expense_purchasers'].purchased_expenses

[PurchasedExpense(expense=Expense(name='groceries'), purchasers=[Person(name='Sarah')]),
 PurchasedExpense(expense=Expense(name='bottle of wine'), purchasers=[Person(name='Sarah')]),
 PurchasedExpense(expense=Expense(name='gas'), purchasers=[Person(name='Jake')]),
 PurchasedExpense(expense=Expense(name='snacks'), purchasers=[Person(name='Jake')]),
 PurchasedExpense(expense=Expense(name='hotel room'), purchasers=[Person(name='Emily')]),
 PurchasedExpense(expense=Expense(name='dinner'), purchasers=[Person(name='Emily')]),
 PurchasedExpense(expense=Expense(name='hotdog'), purchasers=[Person(name='Jake')]),
 PurchasedExpense(expense=Expense(name='popcorn'), purchasers=[Person(name='Jake'), Person(name='Sarah')])]

In [115]:
res['expense_consumers'].consumed_expenses

[ConsumedExpense(expense=Expense(name='groceries'), consumers=[Person(name='Sarah')]),
 ConsumedExpense(expense=Expense(name='bottle of wine'), consumers=[Person(name='Sarah')]),
 ConsumedExpense(expense=Expense(name='gas'), consumers=[Person(name='Jake')]),
 ConsumedExpense(expense=Expense(name='snacks'), consumers=[Person(name='Jake')]),
 ConsumedExpense(expense=Expense(name='hotel room'), consumers=[Person(name='Emily'), Person(name='Sarah')]),
 ConsumedExpense(expense=Expense(name='dinner'), consumers=[Person(name='Emily')]),
 ConsumedExpense(expense=Expense(name='hotdog'), consumers=[Person(name='Jake'), Person(name='Emily')]),
 ConsumedExpense(expense=Expense(name='popcorn'), consumers=[Person(name='Jake'), Person(name='Sarah'), Person(name='Emily')])]

### Figure out how much each person spent on each item
We have who bought what, but not how much they spent on each item

In [116]:
input_to_here = res

{'expense_purchasers': PurchasedExpenses(purchased_expenses=[PurchasedExpense(expense=Expense(name='groceries'), purchasers=[Person(name='Sarah')]), PurchasedExpense(expense=Expense(name='bottle of wine'), purchasers=[Person(name='Sarah')]), PurchasedExpense(expense=Expense(name='gas'), purchasers=[Person(name='Jake')]), PurchasedExpense(expense=Expense(name='snacks'), purchasers=[Person(name='Jake')]), PurchasedExpense(expense=Expense(name='hotel room'), purchasers=[Person(name='Emily')]), PurchasedExpense(expense=Expense(name='dinner'), purchasers=[Person(name='Emily')]), PurchasedExpense(expense=Expense(name='hotdog'), purchasers=[Person(name='Jake')]), PurchasedExpense(expense=Expense(name='popcorn'), purchasers=[Person(name='Jake'), Person(name='Sarah')])]),
 'expense_consumers': ConsumedExpenses(consumed_expenses=[ConsumedExpense(expense=Expense(name='groceries'), consumers=[Person(name='Sarah')]), ConsumedExpense(expense=Expense(name='bottle of wine'), consumers=[Person(name='Sa

In [ ]:
def prep(inp):
    purchased_expenses = inp['expense_purchasers'].purchased_expenses
    consumed_expenses = inp['expense_consumers'].consumed_expenses

In [ ]:
person_amount_per_item_prompt = ChatPromptTemplate.from_template(
"""
The following lists expenses and who purchased them:

Based on the passage below, determine the total for each expense
and the amount each buyer spent on it.

Passage:
{input}

Please calculate how much each person paid for each item.
You are very bad at math, so use the math tools provided.
"""
)

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").with_structured_output(
    Expenses
)

tagging_chain = tagging_prompt | llm

Define some math tools to give the model

In [14]:
from langchain_core.tools import tool

@tool
def sum(floats: List[float]) -> float:
    """Sums a list of floats"""
    return sum(floats)

@tool
def add(first_float: float, second_float: float) -> float:
    """Adds two floats"""
    return first_float + second_float

@tool
def subtract(first_float: float, second_float: float) -> float:
    """Subtracts first_float from second_float"""
    return first_float - second_float

@tool
def divide(dividend: float, divisor: int) -> float:
    """Divides dividend by divisor."""
    return dividend / divisor

@tool
def multiply(float_factor: float, int_factor: int) -> float:
    """Multiplies float_factor by int_factor."""
    return float_factor * int_factor

tools = [sum, add, subtract, divide, multiply]

In [35]:
llm_with_tools = llm.bind_tools(tools)
llm_with_tools_and_structured_output = llm_with_tools.with_structured_output(Expenses)
